In [2]:
import numpy as np
from scipy.stats import ortho_group
from scipy.linalg import fractional_matrix_power
from numpy.linalg import norm
import matplotlib.pyplot as plt
import pandas as pd
import os
import time

In [ ]:
#Data generation

In [3]:
np.random.seed(100)

m, d = 70, 70

# A0 = np.random.normal(mu, var, size=(m, d))
# A = A0 / norm(A0, 2)

D = np.zeros((m, m))
for i in range(0, len(D)):
    D[i, i] = 1 / ((i + 1) ** 4)

U = ortho_group.rvs(dim=m)
A0 = U @ D @ U.T
A = A0 / norm(A0, 2)

W = A.T @ A

z0 = np.random.randint(1, 26, d)
beta = 3  # varz
tau = 5  # noise model. This is very important

# training set of input/outputs


def training_set(n, tau, alpha):
    z_train = np.zeros((n, d))
    for i in range(n):
        z_train[i] = z0 + np.random.normal(0, beta, d)

    x0_train = fractional_matrix_power(W, alpha) @ z_train.T
    x_train = np.zeros((d, n))
    for i in range(n):
        x_train[:, i] = x0_train[:, i] / norm(x0_train[:, i], np.inf)

    eps_train = np.random.normal(0, tau, (m, n))

    y_train = A @ x_train + eps_train

    return x_train, y_train

# When comparing with QO criterion

# alpha0 = 10 ** (57/49)
# q = 10 ** (-8/49)  # In this way lamb\in[10^{-7}, 10]
# N = 50
# lamb = np.zeros(N)
# for i in range(len(lamb)):
#     lamb[i] = alpha0 * (q ** (i+1))


lamb = np.logspace(-7, 1, num=30)
t1 = np.arange(1, 81)

n_it = 30
alph_lan = 3
alph_tik = 0.5
# Full training set
x_tik, y_tik = training_set(10000, tau, alph_tik)

x_lan, y_lan = training_set(10000, tau, alph_lan)

In [ ]:
# Functions

# Tikhonov filter
def tikh(y, la):
    return np.linalg.solve(W + la * np.identity(d), A.T @ y)


# Landweber filter:
def landweber(y, t):
    landw = np.zeros(d)
    iterates = np.zeros((t, d))
    Y = A.T @ y
    for i in range(t):
        landw -= 0.2 * (W @ landw - Y)
        iterates[i] = landw
    return landw, iterates


# Errors for the whole training set
def train_tik(la):  # This function will only be useful to compute lambda*
    ftiklist = np.zeros((len(x_tik.T), d))
    for i in range(len(x_tik.T)):
        ftiklist[i] = tikh(y_tik.T[i], la)
    sumvec = norm(ftiklist - x_tik.T, axis=1) ** 2
    return np.mean(sumvec)


def train_landw(max_t):
    errors = np.zeros((len(x_lan), max_t))
    for i in range(len(x_lan)):
        all_iterates = landweber(y_lan.T[i], max_t)[1]
        errors[i] = norm(all_iterates - x_lan.T[i], axis=1) ** 2
    return np.mean(errors, axis=0)


# Errors depending on the number of training points
def train_tikn(n, la):
    x_tr, y_tr = training_set(n, tau, alph_tik)

    ftiklist = np.zeros((n, d))
    for i in range(n):
        ftiklist[i] = tikh(y_tr.T[i], la)
    sumvec = norm(ftiklist - x_tr.T, axis=1) ** 2
    return np.mean(sumvec)


def train_landwn(n, max_t):
    x_tr, y_tr = training_set(n, tau, alph_lan)
    errors = np.zeros((n, max_t))
    for i in range(n):
        _, all_iterates = landweber(y_tr.T[i], max_t)
        errors[i] = norm(all_iterates - x_tr.T[i], axis=1) ** 2
    return np.mean(errors, axis=0)


# Functions for varynoise, tstartau. n_tr = 100
def tik_tau(tau, lamb, alph):
    n_tr = 100
    xtr, ytr = training_set(n_tr, tau, alph)
    ftiklist = np.zeros((n_tr, d))
    for i in range(n_tr):
        ftiklist[i] = tikh(ytr.T[i], lamb)
    meann = norm(ftiklist-xtr.T, axis=1) ** 2
    return np.mean(meann)


def best_lam(tau, la, alph):  # la is a vector here
    tik_err = np.zeros(len(la))
    for i in range(len(la)):
        tik_err[i] = tik_tau(tau, la[i], alph)
    return la[np.argmin(tik_err)], tik_err.min()


def landw_tau(tau, max_t, alph):
    n_tr = 100
    x_tr, y_tr = training_set(n_tr, tau, alph)
    errors = np.zeros((n_tr, max_t))
    for i in range(n_tr):
        _, all_iterates = landweber(y_tr.T[i], max_t)
        errors[i] = norm(all_iterates - x_tr.T[i], axis=1) ** 2

    return np.mean(errors, axis=0)


def best_t(tau, t, alph):  # t is a vector here
    landw_err = landw_tau(tau, len(t), alph)
    return t[np.argmin(landw_err)], landw_err.min()


# Cross-validation functions

# Tikhonov
def cvlambda(n, lamb):
    tik_err = np.zeros(len(lamb))
    for i in range(len(lamb)):
        tik_err[i] = train_tikn(n, lamb[i])
    return tik_err.min(), lamb[np.where(tik_err == tik_err.min())]


def get_lambda_star(lamb):
    tik_err = np.zeros(len(lamb))
    for i in range(len(lamb)):
        tik_err[i] = train_tik(lamb[i])
    lambda_star = lamb[np.argmin(tik_err)]
    return lambda_star, tik_err.min()


# def get_lambda_hat(L_lambda_star, lamb, N_vec, n_it):
#     Deltan = np.zeros((len(N_vec), n_it))
#     lamb_hatn = np.zeros((len(N_vec), n_it))
#     for i in range(len(N_vec)):
#         for j in range(n_it):
#             lamb_hatn[i, j] = cvlambda(N_vec[i], lamb)[1]
#             Deltan[i, j] = np.abs(L_lambda_star - cvlambda(N_vec[i], lamb)[0])
#     return Deltan, lamb_hatn

def get_lambda_hat(lamb, N_vec, n_it):
    lamb_hatn = np.zeros((len(N_vec), n_it))
    for i in range(len(N_vec)):
        for j in range(n_it):
            lamb_hatn[i, j] = cvlambda(N_vec[i], lamb)[1]
    return lamb_hatn


# Landweber
def cvt(n, t):
    errors = train_landwn(n, len(t))
    return errors.min(), t[np.argmin(errors)]


def get_t_star(t):
    landw_err = train_landw(len(t))
    return t[np.argmin(landw_err)], landw_err.min()


# def get_t_hat(L_t_star, t, N_vec, n_it):
#     Deltan = np.zeros((len(N_vec), n_it))
#     t_hatn = np.zeros((len(N_vec), n_it))
#     for i in range(len(N_vec)):
#         for j in range(n_it):
#             t_hatn[i, j] = cvt(N_vec[i], t)[1]
#             Deltan[i, j] = np.abs(L_t_star - cvt(N_vec[i], t)[0])
#     return Deltan, t_hatn

def get_t_hat(t, N_vec, n_it):
    t_hatn = np.zeros((len(N_vec), n_it))
    for i in range(len(N_vec)):
        for j in range(n_it):
            t_hatn[i, j] = cvt(N_vec[i], t)[1]
    return t_hatn

In [ ]:
# figure 1

np.random.seed(100)

t_EmR = np.arange(1, 1000)
lamb_EmR = np.logspace(-3, 3, num=50)


tik_err = np.zeros((len(lamb_EmR), n_it))
for i in range(len(lamb_EmR)):
    for j in range(n_it):
        tik_err[i, j] = train_tikn(1000, lamb_EmR[i])

meantik = np.mean(tik_err, axis=1)
lowertik = np.quantile(tik_err, 0.05, axis=1)
uppertik = np.quantile(tik_err, 0.95, axis=1)


landw_err = np.zeros((n_it, len(t_EmR)))
for i in range(n_it):
    landw_err[i] = train_landwn(1000, len(t_EmR))

meanlandw = np.mean(landw_err, axis=0)
lowerlandw = np.quantile(landw_err, 0.05, axis=0)
upperlandw = np.quantile(landw_err, 0.95, axis=0)


In [ ]:
# Plots

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5), dpi=100)
# fig.suptitle('Filters')
ax1.semilogx(lamb_EmR, meantik, label='Tikhonov')
ax1.fill_between(lamb_EmR, lowertik, uppertik, alpha=0.2)
ax1.xaxis.set_tick_params(labelsize=15)
ax1.yaxis.set_tick_params(labelsize=15)
ax1.legend(fontsize=15)

ax2.semilogx(t_EmR, meanlandw, label='Landweber')
ax2.fill_between(t_EmR, lowerlandw, upperlandw, alpha=0.2)
ax2.xaxis.set_tick_params(labelsize=15)
ax2.yaxis.set_tick_params(labelsize=15)
ax2.legend(fontsize=15)

fig.supxlabel(r'$\lambda$', fontsize=20)
fig.supylabel(r'$\widehat{L}(f_\lambda)$', fontsize=20)
plt.tight_layout()

plt.savefig("./filters.pdf", bbox_inches='tight')
plt.show(block=False)

In [ ]:
# figure 2

np.random.seed(100)

st = time.time()

tauv = np.logspace(-6, -1, num=50)
tauvec = tauv[::-1]

Lstartik1 = np.zeros((len(tauvec), n_it))
Lstarland1 = np.zeros((len(tauvec), n_it))

Lstartik3 = np.zeros((len(tauvec), n_it))
Lstarland3 = np.zeros((len(tauvec), n_it))

for i in range(len(tauvec)):
    for j in range(n_it):
        Lstartik1[i, j] = best_lam((tauvec[i]), lamb, 0.5)[1]  # case s=0.5
        Lstarland1[i, j] = best_t(tauvec[i], t1, 1)[1]  # case s=1

        Lstartik3[i, j] = best_lam((tauvec[i]), lamb, 0.9)[1]  # case s=0.9
        Lstarland3[i, j] = best_t(tauvec[i], t1, 3)[1]  # case s=3

elapsed_time = time.time() - st
print('Execution time:', time.strftime("%H:%M:%S",
                                       time.gmtime(elapsed_time)))
meantik1 = np.mean(Lstartik1, axis=1)
lowertik1 = np.quantile(Lstartik1, 0.05, axis=1)
uppertik1 = np.quantile(Lstartik1, 0.95, axis=1)

meanland1 = np.mean(Lstarland1, axis=1)
lowerland1 = np.quantile(Lstarland1, 0.05, axis=1)
upperland1 = np.quantile(Lstarland1, 0.95, axis=1)

meantik3 = np.mean(Lstartik3, axis=1)
lowertik3 = np.quantile(Lstartik3, 0.05, axis=1)
uppertik3 = np.quantile(Lstartik3, 0.95, axis=1)

meanland3 = np.mean(Lstarland3, axis=1)
lowerland3 = np.quantile(Lstarland3, 0.05, axis=1)
upperland3 = np.quantile(Lstarland3, 0.95, axis=1)
                                      

In [ ]:
# Plots

plt.close('all')

fig, ax1 = plt.subplots(figsize=(20, 10), dpi=100)
# fig.suptitle("Theorem 3.1")

ax1.plot(tauvec, meantik1, '-', label=r'$s=0.5$ (Tikhonov)')
ax1.plot(tauvec, meantik3, '-', label=r'$s=0.9$ (Tikhonov)')
ax1.fill_between(tauvec, lowertik1, uppertik1, alpha=0.2)
ax1.fill_between(tauvec, lowertik3, uppertik3, alpha=0.2)
ax1.plot(tauvec, meanland1, linestyle='--', label=r'$s=1$ (Landweber)')
ax1.plot(tauvec, meanland3, linestyle='--', label=r'$s=3$ (Landweber)')
ax1.fill_between(tauvec, lowerland1, upperland1, alpha=0.2)
ax1.fill_between(tauvec, lowerland3, upperland3, alpha=0.2)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylabel(r'$L(f_{\lambda^*(\tau)})$', fontsize=20)
ax1.set_xlabel(r"$\tau$", fontsize=20)
ax1.legend(fontsize=20)  # maybe 15?

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.tight_layout()
plt.savefig("./Lstartau.pdf", bbox_inches='tight')
plt.show(block='False')
      

In [ ]:
# Figure 3

np.random.seed(100)

N_vec = np.arange(10, 160, 10)

if 'Deltatik.npy' not in os.listdir():
    # Tikhonov
    lambda_star, L_lambda_star = get_lambda_star(lamb)
    Deltatik, lamb_hatn = get_lambda_hat(L_lambda_star, lamb, N_vec, n_it)

    # Landweber
    t_star, L_t_star = get_t_star(t1)
    Deltalan, t_hatn = get_t_hat(L_t_star, t1, N_vec, n_it)

    np.save('Deltatik.npy', Deltatik)
    np.save('Deltalan.npy', Deltalan)
    np.save('lamhat.npy', lamb_hatn)
    np.save('that.npy', t_hatn)

else:
    Deltatik = np.load('Deltatik.npy')
    Deltalan = np.load('Deltalan.npy')
    lamb_hatn = np.load('lamhat.npy')
    t_hatn = np.load('that.npy')

dftik = pd.DataFrame(Deltatik)
meanDeltik = dftik.mean(axis='columns')
lowertik = np.quantile(Deltatik, 0.05, axis=1)
uppertik = np.quantile(Deltatik, 0.95, axis=1)

dfland = pd.DataFrame(Deltalan)
meanDelland = dfland.mean(axis='columns')
lowerland = np.quantile(Deltalan, 0.05, axis=1)
upperland = np.quantile(Deltalan, 0.95, axis=1)

In [ ]:
# Plots

plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5), dpi=100)

ax1.plot(N_vec, meanDeltik, '-', label='Tikhonov')
ax1.fill_between(N_vec, lowertik, uppertik, alpha=0.2)
ax1.xaxis.set_tick_params(labelsize=15)
ax1.yaxis.set_tick_params(labelsize=15)
ax1.legend(fontsize=15)

ax2.plot(N_vec, meanDelland, linestyle='--', label='Landweber')
ax2.fill_between(N_vec, lowerland, upperland, alpha=0.2)
ax2.xaxis.set_tick_params(labelsize=15)
ax2.yaxis.set_tick_params(labelsize=15)
ax2.legend(fontsize=15)

fig.supxlabel(r'$n_{tr}$', fontsize=20)
fig.supylabel(r'$\Delta(n)$', fontsize=20)
plt.tight_layout(pad=2, w_pad=1.2, h_pad=0.8)
plt.savefig("./ER_paper.pdf", bbox_inches='tight')  # This shud go before show
plt.show(block=False)

In [ ]:
# Figure 4
np.random.seed(100)

st = time.time()

tauvec = np.logspace(-2, 1, num=100)
if 'matrixlam.npy' not in os.listdir():
    lamb_matrix = np.zeros((len(tauvec), n_it))
    t_matrix = np.zeros((len(tauvec), n_it))
    for i in range(len(tauvec)):
        for j in range(n_it):
            lamb_matrix[i, j] = best_lam(tauvec[i], lamb, alph_tik)[0]
            t_matrix[i, j] = best_t(tauvec[i], t1, alph_lan)[0]
    np.save('matrixt.npy', t_matrix)
    np.save('matrixlam.npy', lamb_matrix)
else:
    lamb_matrix = np.load('matrixlam.npy')
    t_matrix = np.load('matrixt.npy')

    elapsed_time = time.time() - st

print('Execution time:', time.strftime("%H:%M:%S",
                                       time.gmtime(elapsed_time)))

meantik = np.mean(lamb_matrix, axis=1)
lowertik = np.quantile(lamb_matrix, 0.05, axis=1)
uppertik = np.quantile(lamb_matrix, 0.95, axis=1)

meanland = np.mean(t_matrix, axis=1)
lowerland = np.quantile(t_matrix, 0.05, axis=1)
upperland = np.quantile(t_matrix, 0.95, axis=1)


In [ ]:
# Plots

plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5), dpi=100)
# fig.suptitle("Behaviour of $\lambda$ varying the noise level")

ax1.plot(tauvec, meantik, '-', label='Tikhonov')
ax1.fill_between(tauvec, lowertik, uppertik, alpha=0.2)
ax1.xaxis.set_tick_params(labelsize=15)
ax1.yaxis.set_tick_params(labelsize=15)
ax1.set_xscale('log')
ax1.legend(fontsize=15)

ax2.plot(tauvec, meanland, linestyle='--', label='Landweber')
ax2.fill_between(tauvec, lowerland, upperland, alpha=0.2)
ax2.xaxis.set_tick_params(labelsize=15)
ax2.yaxis.set_tick_params(labelsize=15)
ax2.set_xscale('log')
ax2.legend(fontsize=15)

fig.supylabel(r'$\hat{\lambda}(\tau)$', fontsize=20)
fig.supxlabel(r'$\tau$', fontsize=20)
plt.tight_layout(pad=2, w_pad=1.2, h_pad=0.8)
# fig.tight_layout()

plt.savefig("./varynoise.pdf", bbox_inches='tight')
plt.show(block=False)

In [ ]:
# Comparison with the Quasi Optimality criterion

def qopt_tikhonov(y, lamda):
    tikdif = np.zeros(len(lamda)-1)
    for i in range(len(tikdif)):
        tikdif[i] = norm(tikh(y, lamda[i])-tikh(y, lamda[i+1]))
    pos = np.where(tikdif == tikdif.min())
    return lamda[pos], tikdif


def qopt_landweber(y):
    landwdif = np.zeros(len(t1))
    iterates = landweber(y, len(t1))[1]
    for i in range(len(landwdif)):
        landwdif[i] = norm(iterates[i] - landweber(A @ iterates[i], i+1)[0])
    pos = np.argmin(landwdif)
    return t1[pos]


N_test = 10
x_ttik, y_ttik = training_set(N_test, tau, alph_tik)
x_tlan, y_tlan = training_set(N_test, tau, alph_lan)


def test_tik(la):
    ftiklist = np.zeros((N_test, d))
    for i in range(N_test):
        ftiklist[i] = tikh(y_ttik.T[i], la)
    meanvec = norm(ftiklist - x_ttik.T, axis=1) ** 2
    return np.mean(meanvec, axis=0)


def test_landw(max_t):
    errors = np.zeros((N_test, max_t))
    for i in range(N_test):
        _, all_iterates = landweber(y_tlan.T[i], max_t)
        errors[i] = norm(all_iterates - x_tlan.T[i], axis=1) ** 2
    return np.min(np.mean(errors, axis=0))


In [ ]:
N_vec = [100]

# Import cvlambhat and cvthat

lamb_hatn = get_lambda_hat(lamb, N_vec, n_it)
lamb_hat_opt = lamb_hatn[0]

t_hatn = get_t_hat(t1, N_vec, n_it)
t_hat_opt = t_hatn[0]


qopt_tik = np.zeros(N_test)
f_qopt_tik = np.zeros((N_test, d))

qopt_land = np.zeros(N_test)
f_qopt_land = np.zeros((N_test, d))

for i in range(N_test):
    qopt_tik[i] = qopt_tikhonov(y_ttik.T[i], lamb)[0]
    f_qopt_tik[i] = tikh(y_ttik.T[i], qopt_tik[i])

    qopt_land[i] = qopt_landweber(y_tlan.T[i])
    f_qopt_land[i] = landweber(y_tlan.T[i], int(qopt_land[i]))[0]

err_qopt_tik = np.mean(norm(f_qopt_tik - x_ttik.T, axis=1) ** 2, axis=0)
err_qopt_land = np.mean(norm(f_qopt_land - x_tlan.T, axis=1) ** 2, axis=0)

crosstik = np.zeros(n_it)
crossland = np.zeros(n_it)
for i in range(n_it):
    crosstik[i] = test_tik(lamb_hat_opt[i])
    crossland[i] = test_landw(int(t_hat_opt[i]))

meantikqopt = np.mean(crosstik - err_qopt_tik)
vartikqopt = np.std(crosstik - err_qopt_tik)

meanlandqopt = np.mean(crossland - err_qopt_land)
varlandqopt = np.std(crossland - err_qopt_land)

print('Quasi-opt case:')
print('Tikhonov mean:', meantikqopt)
print('Tikhonov std:', vartikqopt)
print('Landweber mean:', meanlandqopt)
print('Landweber std:', varlandqopt)

print('Quasiopt Tikhonov params:', qopt_tik)
print('Quasiopt Landw params:', qopt_land)

elapsed_time = time.time() - st

print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))